In [11]:
import pretty_midi
import numpy as np
import os
import IPython
from ast import literal_eval

In [12]:

def bpm_to_120(midi_file):
    """ 
    This function evens out the tempo throuout the song to be 120 BPM,
    even if there are tempo changes. It gets a midi file as an input
    and outputs a modified pretty_midi object
    """
    
    mid = pretty_midi.PrettyMIDI(midi_file)
    
    tempo = mid.get_tempo_changes()
    num_of_changes = len(tempo[0])
    full_length = mid.get_end_time()

    old_times = []
    changes = [0]
    
    for i in range(num_of_changes):
        old_times.append(tempo[0][i])
        if i < (num_of_changes - 1):
            changes.append((tempo[0][i+1] - tempo[0][i]) * (tempo[1][i]/120))
        else:
            changes.append((full_length - tempo[0][i]) * (tempo[1][i]/120))
        
    old_times.append(full_length)
    new_times = np.cumsum(changes)

    mid.adjust_times(old_times, new_times)

    song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    return mid, song_title

In [13]:
import pretty_midi
import os

def extract_guitar_and_drums(mid, song_title='unknown'):
    """This function extracts the guitar and drum tracks from a midi file.
       The input is a path to a midi file (for example: 'raw_data/song_name.mid') in string format
       The output is a dictionary with the song name, guitar track and drum track"""

    if type(mid) == str:
        mid = pretty_midi.PrettyMIDI(mid)
    
    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []
    
    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))

        if (instrument.program >= 25) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))

    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]

    # song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    

        
    song_dict = {'title': song_title,
                 'down_beats': mid.get_downbeats(),
                 'guitar': guitar_track,
                 'drums': drum_track
                }
    return song_dict

In [14]:
#First step

import os

new_directory = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/data/"

midi_files = []

for filename in os.listdir(new_directory):
    full_path = os.path.join(new_directory, filename)
    midi_files.append(full_path)

In [ ]:
# Second step

list_of_midis = []

for song in midi_files:
    list_of_midis.append(bpm_to_120(song))

/home/sergi_carapuig/.pyenv/versions/3.10.6/envs/BandIt/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/home/sergi_carapuig/.pyenv/versions/3.10.6/envs/BandIt/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:1072: UserWarning: original_times must be strictly increasing; automatically enforcing this.
  warnings.warn('original_times must be strictly increasing; '


In [ ]:
print(len(midi_files))

In [5]:
list_ = []

for file in midi_files:
    dict_ = extract_guitar_and_drums(full_path)
    list_.append(dict_)

In [6]:
print(len(list_))

2199


In [7]:
def tracks_to_bars(song_dict: dict):
    """This function accepts a dictionary as an input with 4 keys: 'title', 'down_beats', 'guitar', 'drums'.
    The function takes the guitar and drums, both pretty_midi instrument objects, and cuts them up into a sequence of individual bars.
    The output is a dictionary that contains the following keys/values: song_title, a list of guitar bars, a list of drum bars, and a list of the song's downbeats
    """
    new_dict={}
    new_dict['song_title']=song_dict['title']
    guitar = song_dict['guitar']
    drums = song_dict['drums']
    down_beats_array = song_dict['down_beats']


    guitar_bars_list = []
    drums_bars_list = []

    for index, start_time in enumerate(down_beats_array):
        try:
            end_time = down_beats_array[index+1]            
            guitar_bar = []
            drums_bar = []
            for guitar_note, drum_note in zip(guitar.notes, drums.notes):
                if (guitar_note.start >= start_time) and (guitar_note.end < end_time):
                    guitar_bar.append(guitar_note)    
                if (drum_note.start >= start_time) and (drum_note.end < end_time):
                    drums_bar.append(drum_note)
            drums_bars_list.append(drums_bar)
            guitar_bars_list.append(guitar_bar)
        except:
            #print("done")
            pass
    new_dict['guitar_bars'] = guitar_bars_list
    new_dict['drum_bars'] = drums_bars_list
    new_dict['down_beats'] = down_beats_array.tolist()
    return new_dict

In [ ]:
new_list = []

for e in list_:
    new_list.append(tracks_to_bars(e))